In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
import pdb
import _pickle as cPickle
import time
import subprocess
import glob

import random
import numpy as np
import pandas as pd
import tensorflow as tf

from collections import defaultdict, Counter
from ncrp import Topic, Doc, init, sample, get_perplexity, get_topic_specialization, get_hierarchical_affinities, get_freq_tokens_ncrp, get_docs
from configure import get_config

# load config & data 

In [3]:
config = get_config(nb_name)
np.random.seed(config.seed)
random.seed(config.seed)

In [4]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))

Exception ignored in: <_io.FileIO name='data/20news/instances.pkl' mode='rb' closefd=True>


In [5]:
config.n_doc = len(instances_train)
config.n_vocab = len(bow_idxs)
config.n_doc, config.n_vocab

(11258, 1995)

# run

## initialize log

In [6]:
checkpoint = []
ppl_min = np.inf
epoch = 0

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','VALID:','TEST:','SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','PPL','PPL', 'PPL','1', '2', '3', 'CHILD', 'OTHER']]))))

def update_checkpoint(config, checkpoint, epoch):
    checkpoint.append(config.path_model + '-%i' % epoch)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0)
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize data

In [7]:
topic_root = Topic(idx='0', sibling_idx=0, parent=None, depth=0, config=config)
train_docs = get_docs(instances_train, config)
dev_docs = get_docs(instances_dev, config)
test_docs = get_docs(instances_test, config)
init(train_docs, dev_docs, test_docs, topic_root)

0 10000 0 0 

## run

In [ ]:
time_start = time.time()
while epoch < 100:
# while epoch < config.n_epochs:
    sample(train_docs, dev_docs, test_docs, topic_root)
    ppl_train = get_perplexity(train_docs, topic_root)
    ppl_dev = get_perplexity(dev_docs, topic_root)
    
    if ppl_dev < ppl_min:
        ppl_min = ppl_dev
        ppl_test = get_perplexity(test_docs, topic_root)
        cPickle.dump([test_docs, topic_root], open(config.path_model + '-%i'%epoch, 'wb'))
        update_checkpoint(config, checkpoint, epoch)
        
    depth_spec = get_topic_specialization(test_docs, topic_root)
    hierarchical_affinities = get_hierarchical_affinities(topic_root)
    
    clear_output()
    time_log = int(time.time() - time_start)
    time_start = time.time()
    log_series = pd.Series([time_log, epoch, 0, \
            '%.0f'%ppl_train, '%.0f'%ppl_dev, '%.0f'%ppl_test, \
            '%.2f'%depth_spec[1], '%.2f'%depth_spec[2], '%.2f'%depth_spec[3], \
            '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
            index=log_df.columns)
    log_df.loc[epoch] = log_series    
    display(log_df)
    get_freq_tokens_ncrp(topic_root, idx_to_word, bow_idxs)
    
    cPickle.dump(log_df, open(config.path_log, 'wb'))
    epoch += 1

TRAIN: VALID: TEST: SPEC:             HIER:      
   Time  Ep Ct    PPL    PPL   PPL     1     2     3 CHILD OTHER
0   474   0  0   1031   1030  1035  0.01  0.44  0.44  0.70  0.68
1   546   1  0    957    963   964  0.01  0.46  0.45  0.72  0.71
2   657   2  0    911    927   927  0.01  0.47  0.46  0.76  0.75
3   612   3  0    887    908   908  0.02  0.47  0.46  0.75  0.74
4   599   4  0    867    889   889  0.02  0.47  0.46  0.71  0.70
5   649   5  0    852    872   873  0.03  0.47  0.46  0.73  0.73
6   694   6  0    834    856   857  0.03  0.47  0.46  0.76  0.75
7   674   7  0    818    838   838  0.04  0.46  0.46  0.74  0.73
8   668   8  0    807    826   826  0.04  0.45  0.46  0.74  0.73
9   652   9  0    799    817   817  0.04  0.45  0.46  0.73  0.73
10  663  10  0    795    814   813  0.05  0.45  0.46  0.73  0.73
11  619  11  0    791    810   810  0.05  0.45  0.46  0.67  0.67
12  613  12  0    789    807   807  0.05  0.44  0.46  0.67  0.67
13  623  13  0    787    805   805  0.05  0.44  0.46  0.67  0.66
14  634  14  0    786    804   804  0.05  0.44  0.46  0.70  0.70
15  640  15  0    785    803   802  0.06  0.44  0.46  0.70  0.70
16  635  16  0    784    802   802  0.06  0.44  0.46  0.70  0.69
17  654  17  0    783    801   800  0.06  0.44  0.46  0.70  0.69
18  633  18  0    783    800   800  0.06  0.43  0.46  0.70  0.69
19  640  19  0    783    800   800  0.06  0.43  0.46  0.70  0.69
20  644  20  0    783    800   800  0.06  0.43  0.46  0.70  0.69
21  643  21  0    783    799   799  0.06  0.43  0.46  0.70  0.69
22  633  22  0    782    799   799  0.06  0.43  0.46  0.70  0.69
23  639  23  0    782    798   798  0.06  0.43  0.46  0.70  0.69
24  637  24  0    781    797   798  0.06  0.43  0.46  0.70  0.68
25  641  25  0    781    797   797  0.06  0.43  0.46  0.69  0.68
26  655  26  0    779    796   796  0.06  0.43  0.46  0.69  0.68
27  658  27  0    779    795   795  0.06  0.43  0.46  0.69  0.68
28  656  28  0    778    794   794  0.06  0.43  0.46  0.69  0.68
29  657  29  0    777    794   793  0.06  0.43  0.46  0.69  0.68
..  ...  .. ..    ...    ...   ...   ...   ...   ...   ...   ...
52  662  52  0    761    777   777  0.06  0.44  0.46  0.64  0.63
53  666  53  0    761    777   777  0.06  0.44  0.46  0.64  0.63
54  660  54  0    760    776   775  0.06  0.44  0.46  0.64  0.62
55  658  55  0    759    774   774  0.06  0.44  0.46  0.63  0.62
56  656  56  0    758    773   772  0.06  0.44  0.46  0.63  0.62
57  668  57  0    756    771   771  0.06  0.44  0.46  0.63  0.62
58  691  58  0    755    769   769  0.06  0.44  0.46  0.63  0.62
59  687  59  0    753    766   766  0.06  0.44  0.46  0.62  0.61
60  676  60  0    751    764   764  0.06  0.44  0.46  0.62  0.61
61  676  61  0    748    761   761  0.06  0.45  0.46  0.61  0.60
62  678  62  0    746    758   758  0.06  0.45  0.47  0.61  0.60
63  676  63  0    743    755   754  0.06  0.45  0.47  0.61  0.59
64  677  64  0    740    752   752  0.06  0.45  0.47  0.60  0.59
65  674  65  0    737    749   749  0.06  0.45  0.46  0.60  0.59
66  657  66  0    733    747   746  0.06  0.45  0.47  0.60  0.59
67  656  67  0    730    743   743  0.06  0.46  0.47  0.60  0.59
68  658  68  0    727    741   740  0.06  0.46  0.47  0.60  0.59
69  658  69  0    724    738   738  0.06  0.46  0.47  0.60  0.58
70  655  70  0    721    734   734  0.06  0.46  0.47  0.59  0.58
71  653  71  0    717    731   731  0.06  0.46  0.47  0.59  0.58
72  656  72  0    715    729   729  0.06  0.46  0.47  0.59  0.58
73  660  73  0    713    727   727  0.06  0.46  0.47  0.60  0.59
74  660  74  0    710    725   724  0.07  0.47  0.47  0.60  0.59
75  657  75  0    708    724   723  0.07  0.47  0.47  0.60  0.59
76  656  76  0    706    722   721  0.07  0.47  0.46  0.60  0.59
77  654  77  0    705    720   720  0.07  0.47  0.47  0.60  0.59
78  671  78  0    704    720   719  0.07  0.47  0.46  0.60  0.59
79  684  79  0    702    718   718  0.07  0.47  0.46  0.60  0.59
80  675  80  0    702    717   717  0.07  0.47  0.

 0 11258 382856.0 write one article get know think say like go make
   0-1 2570 141127.0 people government say state gun israel law kill turkish armenian
     0-1-1 1194 7544.0 insurance gun cop safety police car health dog accident officer
     0-1-2 35 133.0 nsa andrew professional domain crack soviet secret little monitor ignore
     0-1-3 924 3386.0 homosexual gay man sexual male sex percent study number publish
     0-1-4 224 417.0 sun earth reality path generation systems half moon next conclude
     0-1-5 18 3.0 boot legitimate change bhj final bob ram dave knowledge prevent
     0-1-6 30 45.0 eye image spot de et left crypto rob detail quality
     0-1-7 145 184.0 nsa ban monitor crack subject many archive exercise appropriate root
   0-2 4206 178318.0 use file drive program get window system run card work
     0-2-1 3962 3525.0 doug engine bike battery voice lock automatic manual brake sell
     0-2-2 17 127.0 ted mon editor frank apr receive id message colorado ad
     0-2-3 

Exception ignored in: <_io.FileIO name='model/20news/ncrp/20news-mncrp-alp10.50.1-eta10-gam0.01-epoch50/log' mode='wb' closefd=True>


0 10000 0 0 